In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [2]:
# create a loss fucntion isntance
loss_function = nn.NLLLoss()
dir(loss_function)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_per

In [23]:
# start with 3 outputs (raw model outputs for 3 tokens in vocab)
#imagine the model takes input data and gives 3 ougtputs corres. to 3 diff categories
model_output = torch.tensor([[-1,2.3,.1]],dtype=torch.float64)
print('Raw model outputs:')
print(f' {model_output[0].tolist()}\n')

# NLLLoss expects log softmax model outputs as its inputs
# applying softmax fun to above outputs to get softmax prob and take the log them (LOGSOFTMAX)
logsoftmax_output = F.log_softmax(model_output, dim =-1)

print('Log softmax outputs:')
print(' ', [round(o.item(),2) for o in logsoftmax_output[0] ],'\n')


# check loss for diff targets
for target in range(len(model_output[0])):
    #which output is the target (correct response)?
    target = torch.tensor([target])

    # calculate loss
    theloss = loss_function(logsoftmax_output, target)
    print(f'When correct output is index "{target.item()}", the loss is {theloss.item():.2f}')

Raw model outputs:
 [-1.0, 2.3, 0.1]

Log softmax outputs:
  [-3.44, -0.14, -2.34] 

When correct output is index "0", the loss is 3.44
When correct output is index "1", the loss is 0.14
When correct output is index "2", the loss is 2.34


In [ ]:
# "When correct output is index "0", the loss is 3.44"
# This is the logsoftmax value for index [0]
# bcoz the simlified eqn: is L = -ln(y^) and ln(y^) = -3.44

In [ ]:
# why 2nd line didnot have loss=0 when model had correct predn???
# that is the beauty of softmax
     # its not a greedy "take the highest value" strategy, isntead:
     # all other values (-3.44, -2.34) are also incorprated into sofmtax value for 2.3
     # so model still learns and going to supress the other 2 value and make 2.3 even larger
     # 

In [25]:
model_output[0][0].item()

-1.0

## multi sample losses (for batches)


In [28]:
# lets crteate 4 batches
batch_output = model_output.repeat(4,1)
print(batch_output,'\n')

# but of course we need logsoftmax
logsoftmax_output = F.log_softmax(batch_output,dim=1)
print(logsoftmax_output)

tensor([[-1.0000,  2.3000,  0.1000],
        [-1.0000,  2.3000,  0.1000],
        [-1.0000,  2.3000,  0.1000],
        [-1.0000,  2.3000,  0.1000]], dtype=torch.float64) 

tensor([[-3.4377, -0.1377, -2.3377],
        [-3.4377, -0.1377, -2.3377],
        [-3.4377, -0.1377, -2.3377],
        [-3.4377, -0.1377, -2.3377]], dtype=torch.float64)


In [30]:
# target changes for each batch
targets = torch.tensor([0,1,2,0]) # target response for each row in the batch


In [31]:
loss = loss_function(logsoftmax_output, targets)
print(loss)

tensor(2.3377, dtype=torch.float64)


In [ ]:
# when training in batches, all of the individual losses get avgd together [verified in below cell]
# so batch learning is a form of regularisation


In [32]:
-torch.tensor([-3.4377 + -0.1377 + -2.3377 + -3.4377])/len(targets)

tensor([2.3377])

In [33]:
loss.backward()
#loss func require a gradient
# the model loss_function is not attached to a model or comp graph

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

## simple example in pytorch

In [40]:
# define a weight matrix (requires_grad = True to track gradients)
w=torch.tensor([[-1,.2]],requires_grad=True)

# target category
target = torch.tensor([0])

#optmiser
optimizer = torch.optim.SGD([w],lr=.5)

#iterations
numTrainingIters = 10

#init some variables
allWeights = torch.zeros((numTrainingIters+1,2))
allWeights[0,:] = w.detach()
allLosses = torch.zeros(numTrainingIters)

#training loop
for i in range(numTrainingIters):
    #reset grads
    optimizer.zero_grad()

    #model outputs (simulating a full model fwd pass)
    modeloutput = F.log_softmax(w,dim=1)
    
    #loss
    loss = loss_function(modeloutput,target)
    allLosses[i] = loss.item()
    
    # Gradient Descent
    loss.backward() # calculate gradient of loss w.r.t w
    optimizer.step() # adjust w using SGD

    #store new weights
    allWeights[i+1,:] = w.detach()

    #and print some results
    print(f"Step {i+1:2d}: loss = {loss.item():.3f}, weights = {[round(o.item(),2) for o in allWeights[i+1]]}")


Step  1: loss = 1.463, weights = [-0.62, -0.18]
Step  2: loss = 0.932, weights = [-0.31, -0.49]
Step  3: loss = 0.610, weights = [-0.08, -0.72]
Step  4: loss = 0.427, weights = [0.09, -0.89]
Step  5: loss = 0.319, weights = [0.23, -1.03]
Step  6: loss = 0.252, weights = [0.34, -1.14]
Step  7: loss = 0.206, weights = [0.43, -1.23]
Step  8: loss = 0.174, weights = [0.51, -1.31]
Step  9: loss = 0.150, weights = [0.58, -1.38]
Step 10: loss = 0.132, weights = [0.64, -1.44]
